In [5]:
import torch
import numpy as np
from IPython.core.debugger import set_trace
import imageio
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data
from torch.autograd import Variable
import torch.nn.functional as F
from torchvision import datasets, transforms



import importlib.util


from tkmodel.TwoCUM import TwoCUMfittingConc
from tkmodel.TwoCUM import TwoCUM

## Single Batch and Multiple Batch torch version of TwoCUM model

#### Input true and predicted PK parameters, it then plots the curves and outputs the MSE loss of the 150 datapoints in each curve.

In [14]:
def TwoCUM_torch(E, Fp ,vp, AIF , t):
    Tp=(vp/Fp)*(1-E)
    #Calculate the IRF
    exptTp= torch.exp(-1*t/Tp) #adding dummy variables so it divides properly
    R=exptTp*(1-E) + E

    #Calculate the convolution
    AIF1 = AIF.view(1, 1, -1)
    R = torch.flip(R, (0,)).view(1, 1, -1)
    temp = t[1]*torch.nn.functional.conv1d(AIF1, R, padding = AIF.shape[0]-1).view(-1)
    F = Fp*temp[0:len(t)]
    return F

def loss_fn(outputs, targets):
    #E, Fp, vp
    #time spacings
    t = np.arange(0,366,2.45)
    AIF = torch.from_numpy(np.load("data/AIF.npy"))

    
    #For outputs
    #First calculate the parameter Tp
    E, Fp ,vp = outputs[0], outputs[1], outputs[2]
    F_out = TwoCUM_torch(E, Fp ,vp, AIF, t)

    
    #For targets - copy pasted
    E, Fp ,vp = targets[0], targets[1], targets[2]
    F_targets = TwoCUM_torch(E, Fp ,vp, AIF, t)

    return torch.sum((F_out - F_targets)**2)/F_out.shape[0]



In [21]:
def TwoCUM_batch(E, Fp ,vp, AIF1 , t): 
    batch_size = E.shape[0]
    
    Tp=(vp/Fp)*(1-E)
    #Calculate the IRF
    exptTp= torch.exp(-1*t[:,None]/Tp[None,:]) #adding dummy variables so it divides properly

    R=exptTp*(1-E) + E

    #Calculate the convolution
    R = torch.flip(R, (0,)).T #Reshape to fit the the convolution
    R = torch.unsqueeze(R, 1)
    temp = t[1]*torch.nn.functional.conv1d(AIF1, R, padding = AIF1.shape[2]-1).view(batch_size, -1)
    F = Fp.unsqueeze(-1)*temp[:,0:len(t)] #unsqueeze to match dimensions
    return F

def loss_fn_batch(outputs, targets):
    #E, Fp, vp
    #time spacings
    t = np.arange(0,366,2.45)
    t = torch.tensor(t)
    
    batch_size = outputs[:,0].shape[0]
    AIF = torch.from_numpy(np.load("data/AIF.npy"))
    AIF1 = AIF.view(1, 1, -1) #reshaped for convolution

    
    #For outputs
    #First calculate the parameter Tp
    E, Fp ,vp = outputs[:,0], outputs[:,1], outputs[:,2]
    F_out = TwoCUM_batch(E, Fp ,vp, AIF1, t)

    
    #For targets - copy pasted
    E_true, Fp_true ,vp_true = targets[:,0], targets[:,1], targets[:,2]
    F_targets = TwoCUM_batch(E_true, Fp_true ,vp_true, AIF1, t)

    
    MSE = torch.sum((F_out - F_targets)**2)/F_out.shape[1]
    return MSE




In [28]:
'''
Testing to see if the batch version works just like the single version. 

As you can see the combined loss of the batch version is the sum of the singles version 
on the same values.
'''
outputs, targets = torch.Tensor([[0.5, 2e-4, 0.5],[0.4, 2e-5, 0.3]]), torch.Tensor([[0.2, 2.5e-5, 0.5],[0.1, 6e-5, 0.3]])
outputs1, targets1 = torch.Tensor([0.5, 2e-4, 0.5]), torch.Tensor([0.2, 2.5e-5, 0.5])
outputs2, targets2 = torch.Tensor([0.4, 2e-5, 0.3]), torch.Tensor([0.1, 6e-5, 0.3])

loss1 = loss_fn(outputs1, targets1)
loss2 = loss_fn(outputs2, targets2)

print('Batch')
loss = loss_fn_batch(outputs, targets)
print('batch = 2',loss, loss.shape)
print('batch = 1',loss1, loss1.shape)
print('batch = 1 second',loss2, loss2.shape)

outputs, targets = torch.DoubleTensor([[0.5, 2e-4, 0.5],[0.4, 2e-5, 0.3]]), torch.DoubleTensor([[0.2, 2.5e-5, 0.5],[0.1, 6e-5, 0.3]])
outputs.requires_grad = True
targets.requires_grad = True

torch.autograd.gradcheck(loss_fn_batch, (outputs, targets))

# outputs1, targets1 = outputs.detach().numpy(), targets.detach().numpy()
# plot_PK(outputs, targets)

Batch
batch = 2 tensor(0.0039, dtype=torch.float64) torch.Size([])
batch = 1 tensor(0.0037, dtype=torch.float64) torch.Size([])
batch = 1 second tensor(0.0002, dtype=torch.float64) torch.Size([])


True

## Clean version to mess with and test the gradients.

In [30]:
def TwoCUM_batch_test(E, Fp ,vp, AIF1 , t): 

    batch_size = E.shape[0]
    
    Tp=(vp/Fp)*(1-E)
    #Calculate the IRF
    exptTp= torch.exp(-1*t[:,None]/Tp[None,:]) #adding dummy variables so it divides properly

    R=exptTp*(1-E) + E

    #Calculate the convolution
    R = torch.flip(R, (0,)).T #Reshape to fit the the convolution
    R = torch.unsqueeze(R, 1)
    temp = t[1]*torch.nn.functional.conv1d(AIF1, R, padding = AIF1.shape[2]-1).view(batch_size, -1)
    F = Fp.unsqueeze(-1)*temp[:,0:len(t)] #unsqueeze to match dimensions
    return F

def loss_fn_batch_test(outputs, targets):
    #E, Fp, vp
    #time spacings
    t = np.arange(0,366,2.45)
    t = torch.tensor(t)
    
    batch_size = outputs[:,0].shape[0]
    AIF = torch.from_numpy(np.load("data/AIF.npy"))
    AIF1 = AIF.view(1, 1, -1) #reshaped for convolution

    
    #For outputs
    #First calculate the parameter Tp
    E, Fp ,vp = outputs[:,0], outputs[:,1], outputs[:,2]
    E.register_hook(lambda x: print('E', x))

    F_out = TwoCUM_batch_test(E, Fp ,vp, AIF1 , t)
    
    #For targets - copy pasted
    E_true, Fp_true ,vp_true = targets[:,0], targets[:,1], targets[:,2]
    F_targets = TwoCUM_batch_test(E, Fp ,vp, AIF1 , t)
    
    MSE = torch.sum((F_out - F_targets)**2)/F_out.shape[1]
    
    F_targets.register_hook(lambda x: print('F_targets', x))
    F_out.register_hook(lambda x: print('F_out',x))
    MSE.register_hook(lambda x: print('MSE',x))


    return MSE


outputs, targets = torch.DoubleTensor([[0.5, 2e-4, 0.5],[0.4, 2e-5, 0.3]]), torch.DoubleTensor([[0.2, 2.5e-5, 0.5],[0.1, 6e-5, 0.3]])
outputs.requires_grad = True
targets.requires_grad = True

loss = loss_fn_batch_test(outputs, targets)
loss.backward()

#this always outputs None?
print(loss.grad)


MSE tensor(1., dtype=torch.float64)
F_targets tensor([[-0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
         -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
         -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
         -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
         -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
         -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
         -0., -0., -0., -0., -0., -0.],
        [-0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
         -0

/var/folders/2m/v5d7mznx1v39h5mv6nwjnvxm0000gn/T/ipykernel_95278/2555747494.py:58: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more information.
  print(loss.grad)


# Unchanged version that outputs non-zero grads for at least some stuff

In [40]:

def loss_fn_batch_test1(outputs, targets):
    #E, Fp, vp
    #time spacings
    t = np.arange(0,366,2.45)
    t = torch.tensor(t)
    
    batch_size = outputs[:,0].shape[0]
    AIF = torch.from_numpy(np.load("data/AIF.npy"))
    AIF1 = AIF.view(1, 1, -1) #reshaped for convolution

    
    #For outputs
    #First calculate the parameter Tp
    E, Fp ,vp = outputs[:,0], outputs[:,1], outputs[:,2]
    Tp=(vp/Fp)*(1-E)
    #Calculate the IRF
    exptTp= torch.exp(-1*t[:,None]/Tp[None,:]) #adding dummy variables so it divides properly
    R=exptTp*(1-E) + E
    #Calculate the convolution
    R = torch.flip(R, (0,)).T #Reshape to fit the the convolution
    R = torch.unsqueeze(R, 1)
    temp = t[1]*torch.nn.functional.conv1d(AIF1, R, padding = AIF.shape[0]-1).view(batch_size, -1)

    F_out = Fp.unsqueeze(-1)*temp[:,0:len(t)] #unsqueeze to match dimensions
    
    #For targets - copy pasted
    E_true, Fp_true ,vp_true = targets[:,0], targets[:,1], targets[:,2]
    Tp_true=(vp_true/Fp_true)*(1-E_true)
    #Calculate the IRF
    exptTp_true= torch.exp(-1*t[:,None]/Tp_true[None,:]) #adding dummy variables so it divides properly

    R_true=exptTp_true*(1-E_true) + E_true

    #Calculate the convolution
    R_true = torch.flip(R_true, (0,)).T #Reshape to fit the the convolution
    R_true = torch.unsqueeze(R_true, 1)
    temp_true = t[1]*torch.nn.functional.conv1d(AIF1, R_true, padding = AIF.shape[0]-1).view(batch_size, -1)
    F_targets = Fp_true.unsqueeze(-1)*temp_true[:,0:len(t)] #unsqueeze to match dimensions
    
    MSE = torch.sum((F_out - F_targets)**2)/F_out.shape[1]
    
    F_targets.register_hook(lambda x: print('F_targets', x, '\n'))
    MSE.register_hook(lambda x: print('MSE', x, '\n'))


    return MSE


outputs, targets = torch.DoubleTensor([[0.5, 2e-4, 0.5],[0.4, 2e-5, 0.3]]), torch.DoubleTensor([[0.2, 2.5e-5, 0.5],[0.1, 6e-5, 0.3]])
outputs.requires_grad = True
targets.requires_grad = True

loss = loss_fn_batch_test1(outputs, targets)
loss.backward()
print(loss.grad)

MSE tensor(1., dtype=torch.float64) 

F_targets tensor([[ 6.1253e-07,  8.8254e-07,  1.1523e-06,  3.7805e-07,  8.1961e-07,
         -3.0827e-07, -5.5562e-07,  7.3240e-07,  6.5745e-07,  5.9639e-08,
         -1.8812e-07, -7.8499e-07,  1.6730e-07,  4.3780e-07,  5.3597e-07,
          8.0607e-07, -4.6309e-06, -3.7176e-05, -7.8609e-05, -1.1536e-04,
         -1.4412e-04, -1.6108e-04, -1.7271e-04, -1.8320e-04, -1.9193e-04,
         -2.0708e-04, -2.2246e-04, -2.3471e-04, -2.4982e-04, -2.5956e-04,
         -2.7318e-04, -2.8751e-04, -2.9765e-04, -3.1220e-04, -3.2366e-04,
         -3.3627e-04, -3.4886e-04, -3.5938e-04, -3.7242e-04, -3.8292e-04,
         -3.9570e-04, -4.0639e-04, -4.1962e-04, -4.3051e-04, -4.3855e-04,
         -4.5080e-04, -4.6280e-04, -4.7276e-04, -4.8520e-04, -4.9717e-04,
         -5.0514e-04, -5.1755e-04, -5.2949e-04, -5.4164e-04, -5.5042e-04,
         -5.6209e-04, -5.7193e-04, -5.8267e-04, -5.9272e-04, -6.0320e-04,
         -6.1436e-04, -6.2371e-04, -6.3439e-04, -6.4416e-04, -6.

/var/folders/2m/v5d7mznx1v39h5mv6nwjnvxm0000gn/T/ipykernel_95278/1927246720.py:55: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more information.
  print(loss.grad)
